In [18]:
import pandas as pd
import tarfile
import xml.etree.ElementTree as ET
from xml.dom.minidom import parse, parseString
from collections import Counter
import numpy as np
import re
import csv
from tqdm import tqdm
import csv
import pickle
import matplotlib.pyplot as plt
import time
import os
import copy
from scipy.stats import anderson
import statsmodels.api as sm
from datetime import datetime
import math
from ordered_set import OrderedSet
import category_encoders as ce
from sklearn.preprocessing import RobustScaler
import pickle
from fuzzywuzzy import fuzz

In [19]:
#pandas settings
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [20]:
path = "../3.new_data/tender_data_2020.csv"
df = pd.read_csv(path)

C:\Users\gbolton\AppData\Local\Temp\ipykernel_17776\3895301322.py:2: DtypeWarning: Columns (18,20) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


In [21]:
df.head()

,ID_NOTICE_CAN,TED_NOTICE_URL,YEAR,ID_TYPE,DT_DISPATCH,XSD_VERSION,CANCELLED,CORRECTIONS,B_MULTIPLE_CAE,CAE_NAME,CAE_NATIONALID,CAE_ADDRESS,CAE_TOWN,CAE_POSTAL_CODE,CAE_GPA_ANNEX,ISO_COUNTRY_CODE,ISO_COUNTRY_CODE_GPA,B_MULTIPLE_COUNTRY,ISO_COUNTRY_CODE_ALL,CAE_TYPE,EU_INST_CODE,MAIN_ACTIVITY,B_ON_BEHALF,B_INVOLVES_JOINT_PROCUREMENT,B_AWARDED_BY_CENTRAL_BODY,TYPE_OF_CONTRACT,TAL_LOCATION_NUTS,B_FRA_AGREEMENT,FRA_ESTIMATED,B_FRA_CONTRACT,B_DYN_PURCH_SYST,CPV,MAIN_CPV_CODE_GPA,ID_LOT,ADDITIONAL_CPVS,B_GPA,GPA_COVERAGE,LOTS_NUMBER,VALUE_EURO,VALUE_EURO_FIN_1,VALUE_EURO_FIN_2,B_EU_FUNDS,TOP_TYPE,B_ACCELERATED,OUT_OF_DIRECTIVES,CRIT_CODE,CRIT_PRICE_WEIGHT,CRIT_CRITERIA,CRIT_WEIGHTS,B_ELECTRONIC_AUCTION,NUMBER_AWARDS,ID_AWARD,ID_LOT_AWARDED,INFO_ON_NON_AWARD,INFO_UNPUBLISHED,B_AWARDED_TO_A_GROUP,WIN_NAME,WIN_NATIONALID,WIN_ADDRESS,WIN_TOWN,WIN_POSTAL_CODE,WIN_COUNTRY_CODE,B_CONTRACTOR_SME,CONTRACT_NUMBER,TITLE,NUMBER_OFFERS,NUMBER_TENDERS_SME,NUMBER_TENDERS_OTHER_EU,NUMBER_TENDERS_NON_EU,NUMBER_OFFERS_ELECTR,AWARD_EST_VALUE_EURO,AWARD_VALUE_EURO,AWARD_VALUE_EURO_FIN_1,B_SUBCONTRACTED,DT_AWARD
0,202050,ted.europa.eu/udl?uri=TED:NOTICE:50-2020:TEXT:...,2020,3,29-DEC-19,R209.S3,0,0,N,Studierendenwerk Thüringen Anstalt des öffentl...,NaN,Philosophenweg 22,Jena,07743,A2,DE,DE,N,NaN,6,NaN,Social protection,N,N,N,W,DEG01,N,NaN,N,N,45215214,45215,NaN,NaN,Y,4,1.0,709130.00,709130.00,709130.00,N,OPE,NaN,0,L,NaN,NaN,NaN,N,1,11099702.0,2,NaN,N,N,Metallbauarbeiten,16-082-Ö-BL,NaN,Erfurt,NaN,DE,N,NaN,"2.08 Metallbauarbeiten – Türen, Glasfassade",8.0,NaN,NaN,NaN,NaN,NaN,709130.00,709130.00,N,24-JAN-18
1,202051,ted.europa.eu/udl?uri=TED:NOTICE:51-2020:TEXT:...,2020,3,29-DEC-19,R209.S3,0,0,N,Studierendenwerk Thüringen Anstalt des öffentl...,NaN,Philosophenweg 22,Jena,07743,A2,DE,DE,N,NaN,6,NaN,Social protection,N,N,N,W,DEG03,N,NaN,N,N,45211340,45211,NaN,45211340,Y,4,1.0,2000000.00,2000000.00,2000000.00,N,OPE,NaN,0,L,NaN,NaN,NaN,N,1,11099703.0,2,NaN,N,N,WWB Umpferstedt,NaN,NaN,Umpferstedt,NaN,DE,N,NaN,Erweiterter Rohbau; Studentenwohnhaus in 07743...,4.0,NaN,NaN,NaN,NaN,NaN,2000000.00,2000000.00,N,10-AUG-18
2,202052,ted.europa.eu/udl?uri=TED:NOTICE:52-2020:TEXT:...,2020,3,30-DEC-19,R209.S3,0,0,N,Sąd Okręgowy w Przemyślu,Pl 34,ul. Konarskiego 6,Przemyśl,37-700,A1,PL,PL,N,NaN,8,NaN,Public Order and Safety,N,N,N,W,PL822,N,NaN,N,N,45216112,45216,NaN,45110000---45300000---45400000---45233250,Y,1,1.0,10206233.96,9877571.91,9877571.91,N,OPE,NaN,0,M,60,"Doświadczenie osoby, która zostanie skierowana...",20---20,N,1,11099704.0,NaN,NaN,N,N,Karpat-Bud Spółka z ograniczoną odpowiedzialno...,180066539,Rogoźnica 305,Głogów Małopolski,36-060,PL,Y,NaN,Rozbudowa budynku Sądu Rejonowego w Przemyślu ...,2.0,2.0,0.0,0.0,2.0,10206233.96,9877571.91,9877571.91,Y,06-DEC-19
3,202053,ted.europa.eu/udl?uri=TED:NOTICE:53-2020:TEXT:...,2020,3,29-DEC-19,R209.S3,0,0,N,Studierendenwerk Thüringen Anstalt des öffentl...,NaN,Philosophenweg 22,Jena,07743,A2,DE,DE,N,NaN,6,NaN,Social protection,N,N,N,W,DEG03,N,NaN,N,N,51112000,45000,NaN,NaN,Y,4,1.0,495000.00,495000.00,495000.00,N,OPE,NaN,0,L,NaN,NaN,NaN,N,1,11099705.0,18,NaN,N,N,OVA Camburg,NaN,NaN,Camburg,NaN,DE,N,NaN,Elektroarbeiten Studentenwohnhaus in 07743 Jen...,8.0,NaN,NaN,NaN,NaN,NaN,495000.00,495000.00,N,14-SEP-18
4,202054,ted.europa.eu/udl?uri=TED:NOTICE:54-2020:TEXT:...,2020,3,29-DEC-19,R209.S3,0,0,N,Studierendenwerk Thüringen Anstalt des öffentl...,NaN,Philosophenweg 22,Jena,07743,A2,DE,DE,N,NaN,6,NaN,Social protection,N,N,N,W,DEG03,N,NaN,N,N,45331000,45331,NaN,NaN,Y,4,1.0,735000.00,735000.00,735000.00,N,OPE,NaN,0,L,NaN,NaN,NaN,N,1,11099706.0,19,NaN,N,N,Borz,NaN,NaN,Hermsdorf,NaN,DE,N,NaN,"Neubau Studentenwohnhaus in 07743 Jena, Spitzw...",7.0,NaN,NaN,NaN,NaN,NaN,735000.00,735000.00,N,20-SEP-18


In [24]:
additional_columns = ["ISO_COUNTRY_CODE", "CAE_TYPE", "MAIN_ACTIVITY", "B_ON_BEHALF", "B_INVOLES_JOINT_PROCUREMENT",
             "B_AWARDED_BY_CENTRAL_BODY", "B_FRA_AGREEMENT", "FRA_ESTIMATED",
             "B_EU_FUNDS", "B_RENEWALS", 
             "CRIT_CODE", "CRIT_PRICE_WEIGHT", "B_RECURRENT_PROCUREMENT",
             "NUMBER_OFFERS", "LOTS_NUMBER", "ID_LOT","ID_NOTICE_CAN", "CANCELLED"]

contract_award_price_columns = ["AWARD_VALUE_EURO_FIN_1", "VALUE_EURO", "AWARD_EST_VALUE_EURO"]

emperical_extension_cols = ["TOP_TYPE", "B_ACCELERATED", "TYPE_OF_CONTRACT", "CAE_NAME", "TAL_LOCATION_NUTS",
                            "CPV", "DT_AWARD", "NUMBER_OFFERS", "INFO_ON_NON_AWARD"]

keep_cols = additional_columns + emperical_extension_cols + contract_award_price_columns

B_ACCELERATED can only have a value when procedure is equal to negotiated or restricted or open procedure. If missing assume it is not accelerated. <br>
emperical extension: <br>
subtype is not a thing in TED database <br> 
neither is postalzone province or postalzone municipality. instead, we use NUTS code<br>
winner province is absent but processed in the nuts code <br>
type of processing is capturd in B_ACCELERATED <br>
award result is captured in INFO_ON_NON_AWARD

In [25]:
df = df.drop(columns = [col for col in df.columns if col not in keep_cols])

--------------------------------------------------------
DROP ROWS
--------------------------------------------------------
--------------------------------------------------------

In [26]:
#LIST ALL CANCELLED TENDERS
missing_CANCELLED = df.loc[df["CANCELLED"] == 1].index.tolist()

#LIST ALL ROWS WITH MISSING VALUES FOR MAIN_ACTIVITY
missing_MAIN_ACTIVITY = df.loc[pd.isna(df["MAIN_ACTIVITY"]) == True].index.tolist()

#LIST ALL MISSING ROWS FOR B_ON_BEHALF
missing_awarded_B_ON_BEHALF = df.loc[pd.isna(df["B_ON_BEHALF"]) == True].index.tolist()

#LIST ALL MISSING ROWS FOR B_AWARDED_BY_CENTRAL_BODY
missing_B_AWARDED_BY_CENTRAL_BODY = df.loc[pd.isna(df["B_AWARDED_BY_CENTRAL_BODY"]) == True].index.tolist()

#LIST ALL MISSING ROWS FOR TAL_LOCATION_NUTS
missing_TAL_LOCATION_NUTS= df.loc[pd.isna(df["TAL_LOCATION_NUTS"]) == True].index.tolist()

#LIST MISSING ROWS FOR PROCEDURE TYPE
missing_TOP_TYPE = df.loc[pd.isna(df["TOP_TYPE"]) == True].index.tolist()

#LIST MISSING ROWS FOR B_EU_FUNDS
missing_B_EU_FUNDS = df.loc[pd.isna(df["B_EU_FUNDS"]) == True].index.tolist()

#LIST MISSING ROWS FOR CRIT_CODE
missing_CRIT_CODE = df.loc[pd.isna(df["CRIT_CODE"]) == True].index.tolist()

#LIST MISSING ROWS FOR NO INFORMATION ON AWARDING CONTRACT
missing_INFO_ON_NON_AWARD = df.loc[pd.isna(df["INFO_ON_NON_AWARD"]) == False].index.tolist()

#LIST MISSING CONTRACT AWARD VALUES
missing_AWARD_VALUE_EURO_FIN_1 = df.loc[pd.isna(df["AWARD_VALUE_EURO_FIN_1"]) == True].index.tolist()

#LIST ALL ROWS FOR WHICH THE VALUE FOR NUMBER_OFFERS IS MISSING
missing_NUMBER_OFFERS = df.loc[pd.isna(df["NUMBER_OFFERS"]) == True].index.tolist()

#LIST ALL ROWS FOR WHICH THE VALUE FOR VALUE_EURO IS MISSING
missing_VALUE_EURO = df.loc[pd.isna(df["VALUE_EURO"]) == True].index.tolist()

#LIST ALL ROWS FOR WHICH THE VALUE FOR AWARD_EST_VALUE_EURO IS MISSING
missing_AWARD_EST_VALUE_EURO = df.loc[pd.isna(df["AWARD_EST_VALUE_EURO"]) == True].index.tolist()

#LITS ALL ROWS FOR WHICH THE VALUE FOR DT_AWARD IS MISSING 
missing_DT_AWARD = df.loc[pd.isna(df["DT_AWARD"]) == True].index.tolist()

#COMBINE ALL LISTS
drop_rows_indices = (missing_CANCELLED + missing_MAIN_ACTIVITY + missing_awarded_B_ON_BEHALF + 
                     missing_B_AWARDED_BY_CENTRAL_BODY + missing_TAL_LOCATION_NUTS +
                     missing_TOP_TYPE + missing_B_EU_FUNDS + missing_CRIT_CODE + missing_INFO_ON_NON_AWARD + 
                     missing_AWARD_VALUE_EURO_FIN_1 + missing_NUMBER_OFFERS + missing_VALUE_EURO + missing_DT_AWARD + missing_AWARD_EST_VALUE_EURO)

#DROP ALL ROWS WHICH HAVE MISSING VALUES FOR THE LISTS CREATED
df = df.drop(labels = drop_rows_indices, axis = 0).reset_index(drop=True)

In [27]:
pd.isna(df).sum()

ID_NOTICE_CAN                     0
CANCELLED                         0
CAE_NAME                          0
ISO_COUNTRY_CODE                  0
CAE_TYPE                          0
MAIN_ACTIVITY                     0
B_ON_BEHALF                       0
B_AWARDED_BY_CENTRAL_BODY         0
TYPE_OF_CONTRACT                  0
TAL_LOCATION_NUTS                 0
B_FRA_AGREEMENT                   0
FRA_ESTIMATED                231858
CPV                               0
ID_LOT                        80249
LOTS_NUMBER                       0
VALUE_EURO                        0
B_EU_FUNDS                        0
TOP_TYPE                          0
B_ACCELERATED                312840
CRIT_CODE                         0
CRIT_PRICE_WEIGHT            213233
INFO_ON_NON_AWARD            322020
NUMBER_OFFERS                     0
AWARD_EST_VALUE_EURO              0
AWARD_VALUE_EURO_FIN_1            0
DT_AWARD                          0
dtype: int64

--------------------------------------------------------
PREPROCESS LOTS_NUMBER
--------------------------------------------------------
--------------------------------------------------------

In [28]:
#MAKE A NEW COLUMN INDICATING IF A TENDER IS SPLIT INTO LOTS
lots_col = []

for i in range(0, len(df)):
    if df["LOTS_NUMBER"].iloc[i] < 2:
        lots_col.append("N")
    elif df["LOTS_NUMBER"].iloc[i] > 1:
        lots_col.append("Y")
    else:
        continue

df["lots"] = lots_col

--------------------------------------------------------
CLEAN FRAMEWORK AGREEMENT COLS
--------------------------------------------------------
--------------------------------------------------------

we distinguish three cases: <br>
(1) a framework was indicated --> do nothing <br>
(2) no framework was indicated, but one of the indicators is present --> assign framework <br>
(3) value is absent, but one of the indicators is present --> assign framework

In [29]:
#in case of A/KA/AC/KAC and C/KC we assume a framework agreement was used
#define the indicators
indicators = ["A", "KA", "AC", "KAC", "C", "KC"]
#find all cases of instance (2)
indices_assign_framework_case_2 = list(df.loc[(df["B_FRA_AGREEMENT"] == "N") & (
    (df["FRA_ESTIMATED"] == indicators[0]) |
    (df["FRA_ESTIMATED"] == indicators[1]) |
    (df["FRA_ESTIMATED"] == indicators[2]) |
    (df["FRA_ESTIMATED"] == indicators[3]) |
    (df["FRA_ESTIMATED"] == indicators[4]) |
    (df["FRA_ESTIMATED"] == indicators[5])
)].index.values)

#find all cases of instance (3)
indices_assign_framework_case_3 = list(df.loc[(pd.isna(df["B_FRA_AGREEMENT"]) == True) & (
    (df["FRA_ESTIMATED"] == indicators[0]) |
    (df["FRA_ESTIMATED"] == indicators[1]) |
    (df["FRA_ESTIMATED"] == indicators[2]) |
    (df["FRA_ESTIMATED"] == indicators[3]) |
    (df["FRA_ESTIMATED"] == indicators[4]) |
    (df["FRA_ESTIMATED"] == indicators[5])
)].index.values)

#combine the lists
indices_assign_framework = indices_assign_framework_case_2 + indices_assign_framework_case_3

#assign framework agreement for those rows in case 2 or 3
for indice in indices_assign_framework:
    df.at[indice, "B_FRA_AGREEMENT"] == "Y"

print(len(indices_assign_framework_case_2), len(indices_assign_framework_case_3))

51400 0


--------------------------------------------------------
PREPROCESS CPV CODES
--------------------------------------------------------
--------------------------------------------------------

In [30]:
#transform cpv column to 2 letter acronyms
def transform_cpv(df, classification_type = 2):
    values = []
    for i in tqdm(range(0, len(df))):
        value = df["CPV"].iloc[i]
        if pd.isna(value) == False:
            values.append(str(value)[:classification_type])
        else:
            values.append(np.nan)
            print("this should not be happenening")
    df["CPV"] = values
    return df

In [31]:
df = transform_cpv(df)

100%|██████████| 322020/322020 [00:01<00:00, 206691.88it/s]


--------------------------------------------------------
PREPROCESS B_ACCELERATED COLUMN
--------------------------------------------------------
--------------------------------------------------------

In [32]:
df["B_ACCELERATED"].unique()

array([nan, 'Y'], dtype=object)

In [33]:
#IF VALUE MISSING FOR B_ACCELERATED, ASSUME IT IS NOT ACCELERATED
b_accelerated_values = [value if pd.notna(value) else "N" for value in df["B_ACCELERATED"]]
df["B_ACCELERATED"] = b_accelerated_values

--------------------------------------------------------
PREPROCESS CRIT_PRICE_WEIGHT COLUMN
--------------------------------------------------------
--------------------------------------------------------

In [34]:
def preprocess_weight_columns(text: str):

    text = str(text)
    pattern = r'\d+(?:[,.]\d+)?'
    numbers_list = re.findall(pattern, text)
    if len(numbers_list) > 0:
        value = numbers_list[0]
        #the value has a comma, replace and multiply
        if "," in value:
            value = float(value.replace(",", "."))
            if value <= 1.0:
                value = int(float(value) * 100)
            elif float(value) > 100 and float(value) < 1000:
                value = int(float(value) / 100)
            else:
                value = int(value)
        elif "." in value and float(value) <= 1.0:
            value = int(float(value) * 100)
        elif float(value) > 100 and float(value) < 1000:
            value = int(float(value) / 100)
        else:
            value = int(float(value))
    else:
        value = np.nan

    return value

In [35]:
crit_price_clean_col = []
count = 0
indices_invalid_values = []

for i in tqdm(range(0, len(df))):
    if pd.isna(df["CRIT_PRICE_WEIGHT"].iloc[i]) == False:
        value = preprocess_weight_columns(df["CRIT_PRICE_WEIGHT"].iloc[i])
        if value > 100:
            indices_invalid_values.append(i)
        else:
            crit_price_clean_col.append(value)
    else:
        crit_price_clean_col.append(np.nan)

df = df.drop(labels = indices_invalid_values, axis = 0).reset_index(drop=True)
df["CRIT_PRICE_WEIGHT"] = crit_price_clean_col
print("A total of {} values were dropped because they contained invalid and inconvertible values".format(len(indices_invalid_values)))

100%|██████████| 322020/322020 [00:02<00:00, 160847.30it/s]


A total of 128 values were dropped because they contained invalid and inconvertible values


In [36]:
#set the CRIT_PRICE_WEIGHT to 100 when criteria is L
crit_price_weight_col_imputed = []
drop_empty_price_weight = []
median = 60

for i in tqdm(range(0, len(df))):
    if pd.isna(df["CRIT_PRICE_WEIGHT"].iloc[i]) == True and df["CRIT_CODE"].iloc[i] == "L":
        crit_price_weight_col_imputed.append(100)
    elif pd.isna(df["CRIT_PRICE_WEIGHT"].iloc[i]) == False:
        crit_price_weight_col_imputed.append(df["CRIT_PRICE_WEIGHT"].iloc[i])
    elif pd.isna(df["CRIT_PRICE_WEIGHT"].iloc[i]) == True and df["CRIT_CODE"].iloc[i] == "M":
        crit_price_weight_col_imputed.append(median)
        drop_empty_price_weight.append(i)
    else:
        continue

#REPLACE COLUMN WITH NEW VALUES AND DROP MISSING ONES
df["CRIT_PRICE_WEIGHT"] = crit_price_weight_col_imputed

100%|██████████| 321892/321892 [00:03<00:00, 88750.71it/s]


--------------------------------------------------------
PREPROCESS DATE COLUMN
--------------------------------------------------------
--------------------------------------------------------

In [37]:
values = []

for i in range(0, len(df)):
    value = df["DT_AWARD"].iloc[i]
    date_time_object = datetime.strptime(value, "%d-%b-%y")
    values.append(date_time_object)

df["DT_AWARD"] = values

--------------------------------------------------------
DROP COLUMNS
--------------------------------------------------------
--------------------------------------------------------

In [38]:
#define list of columns that need to be dropped
drop_cols = ["CANCELLED", "FRA_ESTIMATED", "INFO_ON_NON_AWARD"]
df = df.drop(columns = drop_cols)

In [39]:
pd.isna(df).sum()

ID_NOTICE_CAN                    0
CAE_NAME                         0
ISO_COUNTRY_CODE                 0
CAE_TYPE                         0
MAIN_ACTIVITY                    0
B_ON_BEHALF                      0
B_AWARDED_BY_CENTRAL_BODY        0
TYPE_OF_CONTRACT                 0
TAL_LOCATION_NUTS                0
B_FRA_AGREEMENT                  0
CPV                              0
ID_LOT                       80179
LOTS_NUMBER                      0
VALUE_EURO                       0
B_EU_FUNDS                       0
TOP_TYPE                         0
B_ACCELERATED                    0
CRIT_CODE                        0
CRIT_PRICE_WEIGHT                0
NUMBER_OFFERS                    0
AWARD_EST_VALUE_EURO             0
AWARD_VALUE_EURO_FIN_1           0
DT_AWARD                         0
lots                             0
dtype: int64

In [41]:
with open("../3.new_data/df_from_csv_preprocessed.pickle", "wb") as f:
    pickle.dump(df, f)

In [40]:
len(df)

321892